<a href="https://colab.research.google.com/github/Riteshgupta2906/books/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain-openai langchain langchain-core pydantic langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141

In [5]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field

azure_endpoint = "https://aicoe-dalle3.openai.azure.com/"
azure_deployment = "aicoe-gpt-4o-mini"
azure_api_key = "f7aff489923648a8ae53c6ecabed5214"
azure_openai_api_version = "2023-12-01-preview"

memory = ConversationBufferMemory()
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    api_key=azure_api_key,
    openai_api_version=azure_openai_api_version,
    model_name="gpt-4o-mini",
    temperature=0.7
)



In [6]:
class Weather(BaseModel):
  city: str = Field(
      ...,
      description = "Name of the city"
  )
@tool(args_schema=Weather)
def fetch_weather(city: str):
    """
    Fetches the weather of a given city
    """
    print(city)
    return "Weather in Banglore is 35C"

@tool()
def coffe_shop_details():
    """
    Fetches the details about coffee
    """
    return "Magic is in the beans"

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a witty weather expert that provides the weather for the given city,
            you also provide fun facts about the city.

            """,
        ),
        ("human", "{input}"),
        ("ai", "{agent_scratchpad}"),
    ]
)
tools = [fetch_weather]
agent = create_tool_calling_agent(llm, tools, prompt,memory=memory)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,memory=memory
)

message_input = "Whats the weather in Bangalore"

output = agent_executor.invoke({"input": message_input})
print(output['output'])


Invoking: `fetch_weather` with `{'city': 'Bangalore'}`


Bangalore
Weather in Banglore is 35C
Invoking: `fetch_weather` with `{'city': 'Bangalore'}`


Bangalore
Weather in Banglore is 35CThe weather in Bangalore is a warm 35°C! ☀️

### Fun Facts about Bangalore:
1. **Garden City**: Known as the "Garden City of India," Bangalore boasts numerous parks and gardens, including the famous Lalbagh Botanical Garden.
2. **Silicon Valley of India**: It’s often referred to as the "Silicon Valley of India" due to its booming IT industry and numerous tech startups.
3. **Beer Capital**: Bangalore is known for its vibrant pub culture and is often called the "Pub Capital of India."
4. **Pleasant Climate**: Thanks to its elevation, Bangalore enjoys a moderate and pleasant climate throughout the year.

Stay hydrated and enjoy the sunny vibes! 🌞

> Finished chain.
The weather in Bangalore is a warm 35°C! ☀️

### Fun Facts about Bangalore:
1. **Garden City**: Known as the "Garden City of India," Bangalor

# Day 2


In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

prompt = "you are a chef in a Hotel,try to give a whole procedure to prepare a dish "
question=" How make best chicken Biryani"
input_text = prompt + question
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [11]:
pip install transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain.llms import HuggingFaceLLM
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

ModuleNotFoundError: No module named 'langchain_community'

In [13]:
pip install transformers torch langchain


In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
def generate_response(prompt):
    # Prepare the input text
    input_text = f"Human: {prompt} \nAI:"
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Generate the response
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [16]:
from langchain import Chain
from langchain.prompts import PromptTemplate

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="Human: {user_input} \nAI:"
)

# Define a custom Chain for the chatbot
class T5ChatChain(Chain):
    def __init__(self, model, tokenizer, prompt_template):
        self.model = model
        self.tokenizer = tokenizer
        self.prompt_template = prompt_template

    def _call(self, inputs):
        prompt = self.prompt_template.format(user_input=inputs['user_input'])
        response = generate_response(prompt)
        return {"response": response}

# Instantiate the chat chain
chat_chain = T5ChatChain(model=model, tokenizer=tokenizer, prompt_template=prompt_template)

# Function to get responses from the chat chain
def chat(user_input):
    result = chat_chain({"user_input": user_input})
    return result['response']


ImportError: cannot import name 'Chain' from 'langchain' (/usr/local/lib/python3.10/dist-packages/langchain/__init__.py)

In [17]:
# Install required libraries
!pip install transformers torch langchain

# Import necessary libraries
from transformers import pipeline
from langchain import Chain
from langchain.prompts import PromptTemplate

# Load the FLAN-T5-Large model with the pipeline
pipe = pipeline("text2text-generation", model="google/flan-t5-large")

# Define a function to generate responses using the pipeline
def generate_response(prompt):
    result = pipe(prompt, max_length=150, num_return_sequences=1)
    response = result[0]['generated_text']
    return response

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="Human: {user_input} \nAI:"
)

# Define a custom LangChain class for the chatbot
class T5ChatChain(Chain):
    def __init__(self, pipeline, prompt_template):
        self.pipeline = pipeline
        self.prompt_template = prompt_template

    def _call(self, inputs):
        prompt = self.prompt_template.format(user_input=inputs['user_input'])
        response = generate_response(prompt)
        return {"response": response}

# Instantiate the chat chain
chat_chain = T5ChatChain(pipeline=pipe, prompt_template=prompt_template)

# Function to get responses from the chat chain
def chat(user_input):
    result = chat_chain({"user_input": user_input})
    return result['response']

# Test the chatbot
user_input = "What is the capital of France?"
response = chat(user_input)
print(response)


ImportError: cannot import name 'Chain' from 'langchain' (/usr/local/lib/python3.10/dist-packages/langchain/__init__.py)

In [18]:
# Install required libraries
!pip install transformers torch langchain

# Import necessary libraries
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# Load the FLAN-T5-Large model with the pipeline
pipe = pipeline("text2text-generation", model="google/flan-t5-large")

# Wrap the pipeline with LangChain's HuggingFacePipeline
hf_pipeline = HuggingFacePipeline(pipeline=pipe)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="Human: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=hf_pipeline,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Test the chatbot
user_input = "What is the capital of France?"
response = chat(user_input)
print(response)


ModuleNotFoundError: No module named 'langchain_community'

In [19]:
# Install required libraries
!pip install transformers torch langchain

# Import necessary libraries
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import TransformersLLM

# Load the FLAN-T5-Large model with the pipeline
pipe = pipeline("text2text-generation", model="google/flan-t5-large")

# Wrap the pipeline with LangChain's TransformersLLM
llm = TransformersLLM(pipeline=pipe)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="Human: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Test the chatbot
user_input = "What is the capital of France?"
response = chat(user_input)
print(response)


ModuleNotFoundError: No module named 'langchain_community'

In [28]:


# Import necessary libraries
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Set up the pipeline with the FLAN-T5-Large model
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "max_length": 100,  # Set a maximum length for the generated response
        "temperature": 1,  # Control randomness of the generation
    }
)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a great story teller: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Test the chatbot
user_input = "Tell the thirsty crow story"
response = chat(user_input)
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The thirsty crow is a crow that lives in a crow house. It is a crow that lives in a crow house. It is a crow that lives in a crow house.


In [29]:
# Import necessary libraries
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Set up the pipeline with the FLAN-T5-Large model
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "max_length": 10,  # Set a maximum length for the generated response
        "temperature": 1,  # Control randomness of the generation
    }
)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a great story teller: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Test the chatbot
user_input = "Tell the thirsty crow story"
response = chat(user_input)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The thirsty crow is a


In [30]:
# Import necessary libraries
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Set up the pipeline with the FLAN-T5-Large model
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "max_length": 100,  # Set a maximum length for the generated response
        "temperature": 0,  # Control randomness of the generation
    }
)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a great story teller: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Test the chatbot
user_input = "Tell the thirsty crow story"
response = chat(user_input)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The thirsty crow is a crow that lives in a crow house. It is a crow that lives in a crow house. It is a crow that lives in a crow house.


In [31]:
# Install required libraries
!pip install transformers torch langchain langchain-huggingface

# Import necessary libraries
import tkinter as tk
from tkinter import scrolledtext
from langchain_huggingface import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Set up the pipeline with the FLAN-T5-Large model
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "max_length": 150,
        "temperature": 0.7,
    }
)

# Define a prompt template for the conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="Human: {user_input} \nAI:"
)

# Create a LangChain LLMChain with the prompt template and HuggingFacePipeline
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Function to get responses from the chat chain
def chat(user_input):
    response = chat_chain.run({"user_input": user_input})
    return response

# Create a GUI for the chatbot
class ChatBotApp:
    def __init__(self, root):
        self.root = root
        root.title("Chatbot")

        # Create a scrolled text area for conversation
        self.conversation_area = scrolledtext.ScrolledText(root, state='disabled', wrap='word', height=20, width=50)
        self.conversation_area.pack(padx=10, pady=10)

        # Create an entry box for user input
        self.user_input = tk.Entry(root, width=50)
        self.user_input.pack(padx=10, pady=5)

        # Create a send button
        self.send_button = tk.Button(root, text="Send", command=self.send_message)
        self.send_button.pack(padx=10, pady=5)

    def send_message(self):
        user_text = self.user_input.get()
        if user_text:
            self.conversation_area.config(state='normal')
            self.conversation_area.insert(tk.END, f"You: {user_text}\n")
            self.conversation_area.insert(tk.END, "AI: Thinking...\n")
            self.conversation_area.config(state='disabled')
            self.conversation_area.yview(tk.END)
            self.user_input.delete(0, tk.END)

            # Get AI response
            ai_response = chat(user_text)
            self.conversation_area.config(state='normal')
            self.conversation_area.delete('end-2l', tk.END)  # Remove 'Thinking...' text
            self.conversation_area.insert(tk.END, f"AI: {ai_response}\n")
            self.conversation_area.config(state='disabled')
            self.conversation_area.yview(tk.END)

# Initialize the GUI application
root = tk.Tk()
app = ChatBotApp(root)
root.mainloop()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


TclError: no display name and no $DISPLAY environment variable

In [32]:
# Install required libraries
!pip install transformers torch ipywidgets

# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
from transformers import pipeline

# Load the FLAN-T5-Large model with the pipeline
pipe = pipeline("text2text-generation", model="google/flan-t5-large")

# Create text input widget
text_input = widgets.Text(
    description='Input:',
    placeholder='Type your message here',
    layout=widgets.Layout(width='100%')
)

# Create button widget
submit_button = widgets.Button(description="Generate Response")

# Output area for displaying results
output_area = widgets.Output()

# Define callback function
def on_button_click(b):
    with output_area:
        clear_output()
        user_input = text_input.value
        if user_input:
            # Get model response
            response = pipe(user_input)
            # Display input and response
            print(f"Input: {user_input}")
            print(f"Response: {response[0]['generated_text']}")
        else:
            print("Please enter some text.")

# Bind the button click event to the callback function
submit_button.on_click(on_button_click)

# Display the text input, button, and output area
display(text_input, submit_button, output_area)


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Text(value='', description='Input:', layout=Layout(width='100%'), placeholder='Type your message here')

Button(description='Generate Response', style=ButtonStyle())

Output()